In [52]:
import ee
import geemap
import os
import numpy as np

In [56]:
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [57]:
venus_shp = '/Users/morgansteckler/Downloads/venus_fgmanaus_footprint/venus_fgmanaus_footprint.shp'
manaus_footprint = geemap.shp_to_ee(venus_shp)
Map.addLayer(manaus_footprint, {}, 'Venus Manaus Footprint')

In [58]:
atto_shp = '/Users/morgansteckler/Downloads/venus_atto_footprint/venus_atto_footprint.shp'
atto_footprint = geemap.shp_to_ee(atto_shp)
Map.addLayer(atto_footprint, {}, 'Venus Atto Footprint')

In [59]:
fc = ee.FeatureCollection([manaus_footprint, atto_footprint]).flatten()

In [43]:
#Calculate NDRE and set palette
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)
ndre_params = ({'min': 0.0,'max': 1.0, 'palette' : 
               ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']})

#Create cloud mask
cloudBitMask = ee.Number(2).pow(10).int()
cirrusBitMask = ee.Number(2).pow(11).int()
def maskS2clouds(image):
    qa = image.select('QA60')
    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0)))
    return image.updateMask(mask)

In [44]:
#Load sentinel-2 data and clip to brma_2; set visual params
s2 = (ee.ImageCollection('COPERNICUS/S2') #Select S2 1-C level data
     .filterBounds(fc) #Grab tiles that intersect Manaus, BR, region
     .filterDate(ee.Date('2016-01-01'), ee.Date('2019-12-31')) #Select year I want
     .sort('CLOUDY_PIXEL_PERCENTAGE', False) #Put smallest %cloud image on top of stack
     .map(addNDRE)) #calculate NDRE and add as band

In [45]:
#Create a masked collection
masked = s2.map(maskS2clouds)

In [46]:
#15-day step
years = ee.List.sequence(2016, 2019)
period = ee.List.sequence(1, 365, 15)
col = masked.select('NDRE')

def byYear(y):
    y = ee.Number(y)
    def ndreStepList(startDoy):
        startDoy = ee.Number(startDoy)
        return (col
                .filter(ee.Filter.calendarRange(y, y.add(1), 'year'))
                .filter(ee.Filter.calendarRange(startDoy, startDoy.add(15), 'day_of_year'))
                .qualityMosaic('NDRE'))
    return period.map(ndreStepList)

mapped_doy = years.map(byYear).flatten()
col = ee.ImageCollection.fromImages(mapped_doy)

In [47]:
count = col.size().getInfo()
print(count)

100


In [53]:
# Mean
def region_mean_Image(img):
    # mean over all pixels in the region for the image
    return img.set('mean', img.reduceRegion(ee.Reducer.mean(), geometry=fc, scale=20)) 


def region_mean_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_mean_Image)
    stats_list = stats.aggregate_array('mean')
    return np.array(stats_list.getInfo())

In [54]:
#Create list of dictionaries as [{'NDRE': value} ...] 
mean_list = region_mean_ImageCollection(col)

In [60]:
print(len(mean_list))

100
